In [7]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics
import joblib

In [8]:
# Load Dataset
dataset_name = 'Pima'
df = pd.read_csv('./datasets/diabetes.csv')

In [9]:
# Scale
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])

In [10]:
# Get scaled bounds
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
np.savez(f'models/{dataset_name}_data_bounds.npz', lower_bound=lower_bound, upper_bound=upper_bound)

print(lower_bound, upper_bound)

0.0 1.0


In [5]:
# Check if binary targets
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
df_scaled['target'] = targets

Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


In [6]:
# Train model with 25% of data
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

clf = svm.SVC(kernel='linear')

# Train the model using the training set
clf.fit(X_train, y_train)
joblib.dump(clf, f'models/{dataset_name}_svm_model.pkl')

# Predict the response for test dataset
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.90      0.82       375
           1       0.72      0.46      0.56       201

    accuracy                           0.75       576
   macro avg       0.74      0.68      0.69       576
weighted avg       0.75      0.75      0.73       576



In [7]:
# Finding patterns classified as positive/negative
positive_indexes,negative_indexes = utility.find_indexes(clf, X_test, threshold=0)
print(f"Positive patterns = {len(positive_indexes)},\nNegative patterns = {len(negative_indexes)}")

Positive patterns = 129,
Negative patterns = 447


In [8]:
# Make a dataframe with the test data. For comparing Onestep against Twostep.
test_df_names = list(df.columns)
test_df_names[-1] = 'target' #Change 'Outcome' to 'target'
if 'target' not in test_df_names:
    test_df_names.append('target')
test_dataset = []
for instance, test_class in zip(X_test, y_test.astype('int32')):
    test_dataset.append(np.append(instance, test_class))
test_dataset_df = pd.DataFrame(np.asarray(test_dataset), columns=test_df_names)

In [9]:
# Parameter p value
p = 0.75

#Variables for results
times_twostep = []
rsum_twostep = []
coverage_twostep = []
pos_exp_twostep = []
neg_exp_twostep = []

times_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []

#### Onestep

In [10]:
#Generate Explanations for the patterns classified as negative
for idx in  negative_indexes:  
    #Onestep
    start = time.perf_counter()
    exp = docplex_explainer.onestep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            positive = False)
    end = time.perf_counter()
    times_onestep.append((end - start))
    neg_exp_onestep.append(exp)
    rsum_onestep.append(mymetrics.range_sum(exp))
    coverage_onestep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp)))

#Generate Explanations for the patterns classfied as positive
for idx in positive_indexes:
    #Onestep
    start = time.perf_counter()
    exp = docplex_explainer.onestep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            positive = True)
    end = time.perf_counter()
    times_onestep.append((end - start))
    pos_exp_onestep.append(exp)
    rsum_onestep.append(mymetrics.range_sum(exp))
    coverage_onestep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp)))

#### Twostep

In [11]:
#Generate Explanations for the patterns classified as negative
for idx in  negative_indexes:
    
    #Twostep
    start = time.perf_counter()
    exp_ = docplex_explainer.twostep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            p = p,
            positive = False)
    end = time.perf_counter()
    times_twostep.append((end - start))
    neg_exp_twostep.append(exp_)
    rsum_twostep.append(mymetrics.range_sum(exp_))
    coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))


#Generate Explanations for the patterns classfied as positive
for idx in positive_indexes:
    
    #Twostep
    start = time.perf_counter()
    exp_ = docplex_explainer.twostep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            p = p,
            positive = True)
    end = time.perf_counter()
    times_twostep.append((end - start))
    pos_exp_twostep.append(exp_)
    rsum_twostep.append(mymetrics.range_sum(exp_))
    coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))

#### Metrics

In [12]:
# Compute feature expansion sizes for Twostep
frequency_twostep_neg = utility.detail_exp(
    explanations=neg_exp_twostep, 
    patterns=X_test[negative_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

frequency_twostep_pos = utility.detail_exp(
    explanations=pos_exp_twostep, 
    patterns=X_test[positive_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

neg_sizes_twostep = np.count_nonzero(frequency_twostep_neg.to_numpy() == 1, axis=1)
pos_sizes_twostep = np.count_nonzero(frequency_twostep_pos.to_numpy() == 1, axis=1)
feature_sizes_twostep = np.concatenate([neg_sizes_twostep, pos_sizes_twostep])

In [13]:
# Compute feature expansion sizes for Onestep
frequency_onestep_neg = utility.detail_exp(
    explanations=neg_exp_onestep, 
    patterns=X_test[negative_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

frequency_onestep_pos = utility.detail_exp(
    explanations=pos_exp_onestep, 
    patterns=X_test[positive_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

# Use np.count_nonzero
neg_sizes_onestep = np.count_nonzero(frequency_onestep_neg.to_numpy() == 1, axis=1)
pos_sizes_onestep = np.count_nonzero(frequency_onestep_pos.to_numpy() == 1, axis=1)

# Concatenate directly
feature_sizes_onestep = np.concatenate([neg_sizes_onestep, pos_sizes_onestep])

In [14]:
# Compute means and standard deviations
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

# Compute relative percentage differences
def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

# Ensure all lists are NumPy arrays
times_onestep = np.array(times_onestep)
times_twostep = np.array(times_twostep)
feature_sizes_onestep = np.array(feature_sizes_onestep)
feature_sizes_twostep = np.array(feature_sizes_twostep)
rsum_onestep = np.array(rsum_onestep)
rsum_twostep = np.array(rsum_twostep)
coverage_onestep = np.array(coverage_onestep)
coverage_twostep = np.array(coverage_twostep)

# Compute means and standard deviations
(time_mean_onestep, time_std_onestep) = compute_mean_std(times_onestep)
(time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)

(sizes_mean_onestep, sizes_std_onestep) = compute_mean_std(feature_sizes_onestep)
(sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(feature_sizes_twostep)

(rsum_mean_onestep, rsum_std_onestep) = compute_mean_std(rsum_onestep)
(rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)

(coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(coverage_onestep)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

# Compute relative percentage differences (Mean & Std)
time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_onestep)
sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_onestep)
rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_onestep)
coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)

time_std_diff = relative_percentage_diff(time_std_twostep, time_std_onestep)
sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_onestep)
rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_onestep)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)

# Compute pointwise relative differences
time_relative_pointwise = relative_percentage_diff(times_twostep, times_onestep)
sizes_relative_pointwise = relative_percentage_diff(feature_sizes_twostep, feature_sizes_onestep)
rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_onestep)
coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_onestep)

# Compute pointwise means
time_relative_mean = np.mean(time_relative_pointwise) 
sizes_relative_mean = np.mean(sizes_relative_pointwise)
rsum_relative_mean = np.mean(rsum_relative_pointwise)
coverage_relative_mean = np.mean(coverage_relative_pointwise)

# Compute pointwise standard deviations
time_relative_std = np.std(time_relative_pointwise) 
sizes_relative_std = np.std(sizes_relative_pointwise)
rsum_relative_std = np.std(rsum_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
    'ONESTEP_MEAN': [time_mean_onestep, sizes_mean_onestep, rsum_mean_onestep, coverage_mean_onestep],
    'ONESTEP_STD': [time_std_onestep, sizes_std_onestep, rsum_std_onestep, coverage_std_onestep],
    'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
    'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
    'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
    'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
    'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
    'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std]
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
all_metrics_df.to_csv(f'{dataset_name}/results_{p}.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "times_onestep": times_onestep, 
    "times_twostep": times_twostep,
    "feature_sizes_onestep": feature_sizes_onestep, 
    "feature_sizes_twostep": feature_sizes_twostep,
    "rsum_onestep": rsum_onestep, 
    "rsum_twostep": rsum_twostep,
    "coverage_onestep": coverage_onestep, 
    "coverage_twostep": coverage_twostep,
    "time_relative_%": time_relative_pointwise,
    "sizes_relative_%": sizes_relative_pointwise,
    "rsum_relative_%": rsum_relative_pointwise,
    "coverage_relative_%": coverage_relative_pointwise
})

# Save to CSV
raw_df.to_csv(f"{dataset_name}/raw_metric_data_{p}.csv", index=False)

np.savez(f'{dataset_name}/neg_explanations_{p}.npz', 
         neg_exp_onestep=neg_exp_onestep, 
         neg_exp_twostep=neg_exp_twostep)

# Save positive explanations
np.savez(f'{dataset_name}/pos_explanations_{p}.npz', 
         pos_exp_onestep=pos_exp_onestep, 
         pos_exp_twostep=pos_exp_twostep)

,Metric,ONESTEP_MEAN,ONESTEP_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%
0,Time,0.026331,0.004632,0.036539,0.006641,38.770736,43.394430,39.590442,20.565530
1,Size,5.098958,1.246773,5.098958,1.246773,0.000000,0.000000,0.000000,0.000000
2,Ranges_Sum,4.745900,0.802430,4.764283,0.801824,0.387359,-0.075564,0.397809,0.697762
3,Coverage,12.065972,13.599013,13.024306,13.931483,7.942446,2.444811,14.244322,34.308172
